In [2]:
# !wget https://huggingface.co/datasets/MBZUAI/LaMini-instruction/resolve/main/data/train-00000-of-00003-7899429aebb4446d.parquet
# !wget https://huggingface.co/datasets/MBZUAI/LaMini-instruction/resolve/main/data/train-00001-of-00003-37c47143d7fffd27.parquet
# !wget https://huggingface.co/datasets/MBZUAI/LaMini-instruction/resolve/main/data/train-00002-of-00003-18e21a8292b48a21.parquet

In [3]:
from glob import glob

files = glob('*.parquet')
files

['train-00000-of-00003-7899429aebb4446d.parquet',
 'train-00001-of-00003-37c47143d7fffd27.parquet',
 'train-00002-of-00003-18e21a8292b48a21.parquet']

In [4]:
import pandas as pd

dfs = []
for f in files:
    df = pd.read_parquet(f)
    dfs.append(df)

In [5]:
dfs = pd.concat(dfs)
dfs.head()

,instruction,response
0,Given Miss Robin Crusoe is a 1954 Eastmancolor...,Maybe. While the movie was shot on location in...
1,Combine facts and answer this: Who was the fir...,The first female prime minister of the United ...
2,"@maniar Thanks! I hope this works, that we ca...",positive
3,Write a title:\nROME (CBS) The Arab television...,"""Al-Jazeera Announces Release of Italian Hosta..."
4,Generate a story from key plot points:\n\nJami...,"Jamie Nokes, an 18-year-old, started a craze w..."


In [6]:
html_tags = [
    '<html>',
    '<p>',
    '</a>',
    '<pre>',
    '<code>',
    '<body>',
    '<div>',
]

In [7]:
from tqdm import tqdm

filtered = []
for i in tqdm(range(len(dfs))):
    data = dfs.iloc[i].to_dict()
    if (
        '`' not in data['instruction'] and
        '`' not in data['response'] and
        len(data['instruction']) < 4000 and
        len(data['response']) < 4000 and
        all([t not in data['instruction'] for t in html_tags]) and
        all([t not in data['response'] for t in html_tags])
    ):
        filtered.append(data)

100%|██████████████████████████████| 2582019/2582019 [01:11<00:00, 36211.88it/s]


In [13]:
len(filtered) // 50000

51

In [12]:
import json

with open('lamini-lm-filtered-noncode.jsonl', 'w') as fopen:
    for d in filtered:
        fopen.write(f'{json.dumps(d)}\n')

In [1]:
!split -l 50000 -d --additional-suffix=.splitted lamini-lm-filtered-noncode.jsonl lamini-lm-filtered-noncode.jsonl

In [25]:
filtered = []
for i in tqdm(range(len(dfs))):
    data = dfs.iloc[i].to_dict()
    if (
        ('`' in data['instruction'] or
        '`' in data['response']) and
        len(data['instruction']) < 4000 and
        len(data['response']) < 4000
#         and (any([t in data['instruction'] for t in html_tags]) or
#         any([t in data['response'] for t in html_tags]))
    ):
        filtered.append(data)

100%|██████████████████████████████| 2582019/2582019 [01:04<00:00, 40143.24it/s]


In [26]:
len(filtered)

28895

In [28]:
import json

with open('lamini-lm-filtered-code.jsonl', 'w') as fopen:
    for d in filtered:
        fopen.write(f'{json.dumps(d)}\n')